In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
os.chdir('..')

In [110]:
import random
import torch
import pickle
from decimal import Decimal
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import KFold
from modules.function import get_elem_count, alt_read_gfa_dataset, PTR, check_cuda, get_metrics
from modules.encoder import Encoder1D, EncoderDNN, Encoder
import re
import torch.optim as optim
from torch.utils.data import DataLoader
import tqdm
import joblib
from modules.representation_schemes import get_vectorized_featues, get_atomic_number_features, get_pettifor_features, get_modified_pettifor_features, get_random_features, get_random_features_dense, random_order

In [99]:
random.seed(0)
torch.manual_seed(0)
cuda = check_cuda()

In [6]:
gfa_dataset_file = 'gfa_dataset.txt'
z_row_column_file = 'Z_row_column.txt'
element_property_file = 'element_property.txt'
common_path = "Files_from_GTDL_paper/{}" 
gfa_dataset = pickle.load(open(common_path.format(gfa_dataset_file), 'rb'))  
RC = pickle.load(open(common_path.format(z_row_column_file), 'rb')) 
new_index=[int(i[4]) for i in RC]#new order 
Z_row_column = pickle.load(open(common_path.format(z_row_column_file), 'rb'))
[property_name_list,property_list,element_name,_]=pickle.load(open(common_path.format(element_property_file), 'rb'))

In [7]:
comps_gfa, y, p = alt_read_gfa_dataset()
count_dict = get_elem_count(comps_gfa)
count_df = pd.DataFrame.from_dict(count_dict, orient='index')
count_df.reset_index(inplace=True)
count_df.rename({'index':'element',0:'count'}, axis=1, inplace=True)

In [13]:
count_df['percent'] = [np.round(count_df.loc[i,'count']/len(comps_gfa)*100,2) for i in range(count_df.shape[0])]
elements = ['Al','Ni','Fe','Cr','Co','Ti','Mo','Nb','V','W']
count_df[[el in elements for el in count_df['element'].values]].sort_values('percent')

,element,count,percent
37,W,380,1.82
9,V,500,2.40
21,Nb,1078,5.17
22,Mo,1222,5.86
8,Ti,1930,9.25
13,Co,2294,10.99
10,Cr,2714,13.00
12,Fe,5302,25.40
14,Ni,6112,29.29
4,Al,7084,33.94


In [105]:
elements = ['Al','Ni','Fe','Cr','Co','Ti','Mo','Nb','V','W']
filename = 'misc/gfa_gen_splits.json'
if os.path.exists(filename):
    with open(filename,'rb') as fid:
        gfa_gen_dict = json.load(fid)
        print('Split file loaded')
else:

    gfa_gen_dict = {}
    for el in elements:
        train_inds, test_inds,all_cv_train, all_cv_test = [],[],[],[]
        for i,c in enumerate(comps_gfa):
            if el in c.get_el_amt_dict().keys():
                train_inds.append(i)
            else:
                test_inds.append(i)
        X_train = np.array(comps_gfa)[train_inds]
        kf = KFold(n_splits=5)
        for tr, ts in kf.split(X_train):
            all_cv_train.append(tr.tolist())
            all_cv_test.append(ts.tolist())
        gfa_gen_dict[el] = {'train':train_inds,'test':test_inds, 'cv_train':all_cv_train, 'cv_test':all_cv_test}
    
    with open(filename,'w') as fid:
        json.dump(gfa_gen_dict, fid)
        print('Split file written!')
        

/tmp/ipykernel_260629/291367725.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(comps_gfa)[train_inds]


Split file written!


In [106]:
def get_1D_features_gfa(k:str):
    comp_gfa, y, p = alt_read_gfa_dataset()
    y = np.array(y).reshape(-1,1).astype('float32')
    p = np.array(p).reshape(-1,1).astype('float32')
    if k not in ['atomic','pettifor','mod_pettifor','random']:
        print('Unsupported format')
        return None, None, None
    else:
        if k == 'atomic':
            comp, at_order  = get_atomic_number_features(comp_gfa)
        elif k == 'pettifor':
            comp, _  = get_pettifor_features(comp_gfa)
        elif k == 'mod_pettifor':
            comp, _  = get_modified_pettifor_features(comp_gfa)
        elif k == 'random':
            comp,_ = get_random_features(comp_gfa, random_order)
        return comp, y, p

def get_dense_features_gfa():
    comp_gfa, y, p = alt_read_gfa_dataset()
    y = np.array(y).reshape(-1,1).astype('float32')
    p = np.array(p).reshape(-1,1).astype('float32') 
    comp,_ = get_random_features_dense(comp_gfa, random_order)
    return comp, y, p

def get_ptr_features_gfa(gfa_dataset=gfa_dataset):

    gfa_i=[]
    gfa_a=[]
    gfa_b=[]
    gfa_c=[]
    to_discard = ['Rf','Db','Sg','Bh','Hs']
    for i in  gfa_dataset:
        tx_gfa=re.findall('\[[a-c]?\]', i)
        tx1_element=re.findall('[A-Z][a-z]?', i)#[B, Fe, P,No]
        if len(set(tx1_element).intersection(set(to_discard))) == 0:      
            gfa_i.extend(tx_gfa)
            if tx_gfa[0]=='[a]':
                gfa_a.append(gfa_dataset.index(i))
            elif tx_gfa[0]=='[b]':
                gfa_b.append(gfa_dataset.index(i)) 
            else:
                gfa_c.append(gfa_dataset.index(i))
        
    gfa_data_form=[]
    gfa_data_form_p = []
    gfa_data_form_b=[]

#------------------------------------------------------------------------------
#map raw data to 2-D image using PTR
    for i in gfa_a:
        x,p,y = PTR(gfa_dataset[i])
        gfa_data_form=gfa_data_form+x
        gfa_data_form_p = gfa_data_form_p+p
        gfa_data_form_b=gfa_data_form_b+y
    for i in gfa_c:
        x,p,y = PTR(gfa_dataset[i])
        gfa_data_form=gfa_data_form+x
        gfa_data_form_p = gfa_data_form_p+p
        gfa_data_form_b=gfa_data_form_b+y 
    for i in gfa_b:
        x,p,y = PTR(gfa_dataset[i])
        gfa_data_form=gfa_data_form+x
        gfa_data_form_p = gfa_data_form_p+p
        gfa_data_form_b=gfa_data_form_b+y

    X_all = np.array(gfa_data_form).reshape(-1, 1,9, 18).astype('float32') 
    y_all = np.array(gfa_data_form_b).reshape(-1,1).astype('float32')
    p_all = np.array(gfa_data_form_p).reshape(-1,1).astype('float32')
    return X_all, y_all, p_all

In [111]:
saveloc = 'saved_models/LOEO_Encoders'
if not os.path.exists(saveloc):
    os.makedirs(f'{saveloc}')

In [119]:
result_file = 'results/gfa_LOEO_stats.json'
methods = ['dense','atomic','pettifor','mod_pettifor','random','PTR']
batch = 64
num_iterations = 2000
log_interval = int(5e2)
results_dict = {}
for method in methods:
    results_dict[method] = {}
    if method == 'dense':
        X, y , p = get_dense_features_gfa()    
    elif method in ['atomic','pettifor','mod_pettifor','random']:
        X,y,p = get_1D_features_gfa(method)     
    elif method == 'PTR':
        X,y,p = get_ptr_features_gfa()
    for k in gfa_gen_dict.keys():
        results_dict[method][k] = {}
        f1_max = 0
        best_fold_model = 0
        LOEO_dict = {'Fold_stats':{}}
        test_inds = gfa_gen_dict[k]['test']
        X_test = X[test_inds]
        y_test = y[test_inds]
        p_test = p[test_inds]
        if X_test.dtype != torch.float32:
            X_test = torch.from_numpy(X_test)
        if p_test.dtype != torch.float32:
            p_test = torch.from_numpy(p_test)
        if cuda:
            X_test = X_test.cuda()
            p_test = p_test.cuda()
        cv_train_inds, cv_test_inds = gfa_gen_dict[k]['cv_train'],gfa_gen_dict[k]['cv_test']
        for i in range(len(cv_train_inds)):
            fold_train_inds, fold_test_inds = cv_train_inds[i], cv_test_inds[i]
            X_train_fold,y_train_fold ,p_train_fold  = X[fold_train_inds], y[fold_train_inds], p[fold_train_inds]
            X_test_fold, y_test_fold, p_test_fold = X[fold_test_inds], y[fold_test_inds], p[fold_test_inds]
            Xy = [(X_train_fold[i],y_train_fold[i],p_train_fold[i]) for i in range(len(y_train_fold))]
            train_loader = DataLoader(Xy, batch_size = batch , shuffle=True)
            if X_test_fold.dtype != torch.float32:
                X_test_fold = torch.from_numpy(X_test_fold)
            if p_test_fold.dtype != torch.float32:
                p_test_fold = torch.from_numpy(p_test_fold)
            if method == 'dense':
                encoder = EncoderDNN(X_train_fold.shape[-1],3,42,1)
            elif method in ['atomic','pettifor','mod_pettifor','random']:
                encoder = Encoder1D(1,1)
            elif method == 'PTR':
                encoder = Encoder(1,1)
            
            if cuda:
                encoder = encoder.cuda()
                X_test_fold, p_test_fold = X_test_fold.cuda(), p_test_fold.cuda()
            e_optimizer = optim.Adam(encoder.parameters(),lr = 2e-4)
            for iter in tqdm.notebook.tqdm(range(num_iterations)):
                train_loss = 0.0
                for data in train_loader:
                    X_t,y_t,p_t = data
                    if cuda:
                        X_t = X_t.cuda()
                        y_t = y_t.cuda()
                        p_t = p_t.cuda()
                    e_optimizer.zero_grad()
                    target = encoder(X_t,p_t)
                    if cuda:
                        target = target.cuda()
                    e_error = torch.nn.BCELoss()(target,y_t)
                    e_error.backward(retain_graph=True)
                    e_optimizer.step()
                    train_loss += e_error.cpu().item()
                if iter == 0 or (iter + 1) % log_interval == 0:  
                    print('{} : Element {}, Fold {}, Epoch : {}, Loss : {}'.format(method,k,i,iter+1,train_loss))
            spec_saveloc = os.path.join(saveloc,method)
            if not os.path.exists(spec_saveloc):
                os.makedirs(f'{spec_saveloc}')
            joblib.dump(encoder,os.path.join(spec_saveloc,'LOEOEncoder_{}_fold{}.pt'.format(k,i)))
            y_predict_fold = (encoder(X_test_fold,p_test_fold)).to('cpu').detach().numpy()
            metrics_fold = get_metrics(y_test_fold,np.round(y_predict_fold))
            LOEO_dict['Fold_stats'][i] = metrics_fold
            y_predict = (encoder(X_test,p_test)).to('cpu').detach().numpy()
            metrics = get_metrics(y_test,np.round(y_predict))
            f1_predict = metrics[3]
            if f1_predict>f1_max:
                f1_max = f1_predict
                best_fold_model = i
                LOEO_dict['Best_f1'] = f1_max
                LOEO_dict['Best_fold'] = best_fold_model
        results_dict[method][k] = LOEO_dict
        if os.path.exists(result_file):
            with open(result_file,'rb') as fid:
                data_file = json.load(fid)
            updated_file = data_file|results_dict
            with open(result_file,'w') as f:
                json.dump(updated_file,f)
        else:
            with open(result_file, 'w') as f:
                json.dump(results_dict, f)
        

  0%|          | 0/2000 [00:00<?, ?it/s]

dense : Element Al, Fold 0, Epoch : 1, Loss : 59.46879905462265
dense : Element Al, Fold 0, Epoch : 500, Loss : 5.203987020999193
dense : Element Al, Fold 0, Epoch : 1000, Loss : 4.191072904504836
dense : Element Al, Fold 0, Epoch : 1500, Loss : 3.908863263204694
dense : Element Al, Fold 0, Epoch : 2000, Loss : 3.772492978256196


  0%|          | 0/2000 [00:00<?, ?it/s]

dense : Element Al, Fold 1, Epoch : 1, Loss : 60.659728050231934
dense : Element Al, Fold 1, Epoch : 500, Loss : 4.441414857748896
dense : Element Al, Fold 1, Epoch : 1000, Loss : 3.7315723549108952
dense : Element Al, Fold 1, Epoch : 1500, Loss : 3.4324506116099656
dense : Element Al, Fold 1, Epoch : 2000, Loss : 3.312455483013764


  0%|          | 0/2000 [00:00<?, ?it/s]

dense : Element Al, Fold 2, Epoch : 1, Loss : 57.727927446365356
dense : Element Al, Fold 2, Epoch : 500, Loss : 4.739594083279371
dense : Element Al, Fold 2, Epoch : 1000, Loss : 3.9628490114118904
dense : Element Al, Fold 2, Epoch : 1500, Loss : 4.0655128138605505
dense : Element Al, Fold 2, Epoch : 2000, Loss : 3.1386625161394477


  0%|          | 0/2000 [00:00<?, ?it/s]

dense : Element Al, Fold 3, Epoch : 1, Loss : 51.827803552150726
dense : Element Al, Fold 3, Epoch : 500, Loss : 5.852062711492181
dense : Element Al, Fold 3, Epoch : 1000, Loss : 4.74813527520746
dense : Element Al, Fold 3, Epoch : 1500, Loss : 4.108698435593396
